In [29]:
""" Implementing normal GAN for synthetic data experiment """

import numpy as np

from IPython.core.debugger import Tracer

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Activation, Lambda
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model, Sequential
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from keras.utils import np_utils

import matplotlib.pyplot as plt


from tensorflow import set_random_seed
# for reproducibility 
np.random.seed(1)
set_random_seed(1)

In [36]:

class GAN(object):

    """ Generative Adversarial Network class """
    def __init__(self):

        self.x_combined_batch = None
        self.y_combined_batch = None
        self.number_of_classes = 2
        self.input_dim = 2
        self.gen_input_dim = 100
        self.epochs = 20000
        self.batch_size = 200
        self.save_interval = 100
        # There are 2 inlier circles
        # Inlier circle centers on the 1st axis; for all other dimensions, the centers are at 0.
        self.x1 = 0.4
        self.x2 = 0.61
        self.r = 0.1
        # Number of inlier samples for each class
        self.n = 1000
        # width of the outlier outer square side
        self.s = 100

        # Confidence loss weight
        self.beta = 0.0

        # get in-distribution data from 2 circles (2 classes)
        self.data, self.labels = self.get_data()

        print(self.data.shape, self.labels.shape)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.data, self.labels, test_size=0.5, shuffle=True, stratify=self.labels,
                                                            random_state=1)

        self.Y_train = np_utils.to_categorical(self.y_train, self.number_of_classes)
        self.Y_test = np_utils.to_categorical(self.y_test, self.number_of_classes)

        self.optimizer = Adam(lr=0.0002, beta_1=0.5, decay=8e-8)

        self.G = self.__generator()
        self.G.compile(loss='binary_crossentropy', optimizer=self.optimizer)

        self.D = self.__discriminator()
        self.D.compile(loss='binary_crossentropy', optimizer=self.optimizer, metrics=['accuracy'])

        self.stacked_generator_discriminator = self.__stacked_generator_discriminator()
        self.stacked_generator_discriminator.compile(loss='binary_crossentropy', optimizer=self.optimizer)




    # return True if point p (a list) lies inside a hypersphere centered at the origin with radius r
    def is_in_hypersphere(self, r, p):
        p = np.array(p)
        return np.sum(p*p) <= r**2


    # return True if point p (a list) lies inside a hypersphere centered at the (x, 0, 0, ...) with radius r
    def is_in_hypersphere_centered_at(self, x, r, p):
        p = np.array(p)
        p[0] -= x
        return np.sum(p*p) <= r**2


    # uniformly sample a point from a square centered at the origin with side = s
    def sample_point_from_square(self, s):
        p = []
        # first point
        for i in range(0, self.input_dim):
            p.append(np.random.uniform(-1*s/2, s/2, 1))
        return p


    # sample points using rejection sampling from a sphere centered at the origin
    def sample_point_from_hypersphere(self, r):
        p = self.sample_point_from_square(2*r)
        while not self.is_in_hypersphere(r, p):
            p = self.sample_point_from_square(2*r)
        return p
            

    # sample from a hypersphre with rejection sampling
    def get_hypersphere_data(self, x, r, n):
        data = np.zeros((n, self.input_dim))
        count = 0
        while count<n:
            data[count, :] = self.sample_point_from_hypersphere(r)
            # update the point with offset for the 1st dimension
            data[count, 0] += x
            count += 1

        data = data.astype(np.float32)
        return data
        # plt.scatter(x, y, s = 4)
    
    
    def get_data(self):
        # class 0 data
        data = self.get_hypersphere_data(self.x1, self.r, self.n)
        labels = np.ones((self.n, 1))*0
        print(labels.shape)
        # class 1 data
        data = np.concatenate((data, self.get_hypersphere_data(self.x2, self.r, self.n)), axis=0)
        labels = np.concatenate((labels, np.ones((self.n, 1))*1), axis=0)
        print(labels.shape)

        return data.astype(np.float32), labels.astype(np.float32)


    def __generator(self):
        """ Declare generator """

        model = Sequential()
        model.add(Dense(256, input_shape=(self.gen_input_dim,)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(self.input_dim, activation='linear'))

        return model

    def __discriminator(self):
        """ Declare discriminator """

        model = Sequential()
        model.add(Dense(256, input_shape=(self.input_dim, )))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        return model

    
    """
    def __generator(self):
        # Declare generator 

        model = Sequential()
        model.add(Dense(500, input_shape=(self.gen_input_dim,)))
        model.add(Activation('relu'))
        model.add(Dense(500))
        model.add(Activation('relu'))
        model.add(Dense(self.input_dim, activation='linear'))

        return model

    def __discriminator(self):
        # Declare discriminator

        model = Sequential()
        model.add(Dense(500, input_shape=(self.input_dim, )))
        model.add(Activation('relu'))
        model.add(Dense(500))
        model.add(Activation('relu'))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        return model
    """


    def __stacked_generator_discriminator(self):

        self.D.trainable = False

        model = Sequential()
        model.add(self.G)
        model.add(self.D)

        return model


    def train(self):

        # gen_data = np.empty((self.epochs*10, self.input_dim))
        # count = 0
        y_discriminator_ones = np.ones((self.batch_size/2, 1)).astype(np.float32)
        y_discriminator_zeros = np.zeros((self.batch_size/2, 1)).astype(np.float32)
        y_out = np.ones((self.batch_size/2, 2)).astype(np.float32)
        sample_weight = np.ones((self.batch_size)).astype(np.float32)
        sample_weight[self.batch_size/2:] = self.beta

        for cnt in range(self.epochs):

            ## train discriminator
            random_index = np.random.randint(0, len(self.X_train) - self.batch_size/2)
            legit_images = self.X_train[random_index : random_index + self.batch_size/2]

            gen_noise = np.random.normal(0, 1, (self.batch_size/2, self.gen_input_dim))
            synthetic_images = self.G.predict(gen_noise)

            self.x_combined_batch = np.concatenate((legit_images, synthetic_images))
            self.y_combined_batch = np.concatenate((y_discriminator_ones, y_discriminator_zeros))

            d_loss = self.D.train_on_batch(self.x_combined_batch, self.y_combined_batch)

            """
            gen_noise = np.random.normal(0, 1, (100, 100))
            syntetic_images = self.G.predict(gen_noise)
            preds = self.D.predict(syntetic_images)
            ind = np.where(preds < self.thresh)[0]
            if len(ind) > 0:
                gen_data[count:count+min(10, ind.shape[0]),] = syntetic_images[ind[0:min(10, ind.shape[0])],]
                count = count + min(10, ind.shape[0])
            """

            # train generator
            noise = np.random.normal(0, 1, (self.batch_size/2, self.gen_input_dim))
            g_loss = self.stacked_generator_discriminator.train_on_batch([noise], [y_discriminator_ones])

            print('epoch: %d' % cnt)
            print(d_loss)
            print(g_loss)
            #print ('epoch: %d, [Discriminator :: d_loss: %f], [ Generator :: loss: %f], [ Classifier :: loss: %f]' % 
            #       (cnt, d_loss[0], g_loss, 10.1))

            if cnt % self.save_interval == 0:
                self.plot_data(save2file=True, step=cnt)



    def plot_data(self, save2file, step=0):
        ''' Plot and generated images '''

        gen_noise = np.random.normal(0, 1, (self.X_test.shape[0], self.gen_input_dim))
        synthetic_images = self.G.predict(gen_noise)

        x = np.concatenate((self.X_test, synthetic_images), axis=0)
        #x = self.x_combined_batch
        # display a 2D plot of the digit classes in the latent space

        y = np.concatenate((self.y_test, 2 * np.ones((synthetic_images.shape[0], 1))))
        #y = self.y_combined_batch
        plt.figure(figsize=(12, 10))
        plt.scatter(x[:, 0], x[:, 1], c=y.squeeze())
        plt.colorbar()
        plt.xlabel("x[0]")
        plt.ylabel("x[1]")
        if save2file:
            filename = "./images/mnist_%d.png" % step
            plt.savefig(filename)
        plt.close('all')
        #plt.show()



In [37]:
gan = GAN()
gan.train()

(1000, 1)
(2000, 1)
((2000, 2), (2000, 1))
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_117 (Dense)            (None, 256)               768       
_________________________________________________________________
leaky_re_lu_88 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_118 (Dense)            (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_89 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_119 (Dense)            (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_90 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_120 (Dense)            (Non

epoch: 183
[0.21451122, 0.93]
5.517732
epoch: 184
[0.22177719, 0.93]
5.343207
epoch: 185
[0.2421598, 0.92]
5.8327107
epoch: 186
[0.21228427, 0.95]
5.2427135
epoch: 187
[0.22103581, 0.94]
6.0589314
epoch: 188
[0.23065655, 0.9]
5.4367766
epoch: 189
[0.25914162, 0.905]
6.2035966
epoch: 190
[0.24230987, 0.935]
5.8218822
epoch: 191
[0.28588882, 0.925]
5.5711794
epoch: 192
[0.1855867, 0.945]
5.568935
epoch: 193
[0.2635122, 0.885]
5.514895
epoch: 194
[0.21365513, 0.94]
5.7079167
epoch: 195
[0.24923752, 0.925]
6.7147923
epoch: 196
[0.2821381, 0.915]
5.4682927
epoch: 197
[0.22042198, 0.945]
6.4146338
epoch: 198
[0.22443731, 0.92]
5.8153462
epoch: 199
[0.20738472, 0.945]
6.0671062
epoch: 200
[0.21238935, 0.935]
6.555136
epoch: 201
[0.18936405, 0.945]
4.9228444
epoch: 202
[0.22391804, 0.935]
5.379043
epoch: 203
[0.21845666, 0.955]
5.4360003
epoch: 204
[0.21302381, 0.94]
5.4346137
epoch: 205
[0.27519283, 0.9]
5.9161572
epoch: 206
[0.240995, 0.935]
5.994725
epoch: 207
[0.23793533, 0.92]
5.810538
ep

epoch: 399
[0.33148453, 0.89]
5.7692823
epoch: 400
[0.3435277, 0.87]
5.5326366
epoch: 401
[0.3445979, 0.88]
5.7752724
epoch: 402
[0.39116994, 0.865]
4.914505
epoch: 403
[0.31168014, 0.885]
5.8698716
epoch: 404
[0.36093357, 0.87]
5.1236897
epoch: 405
[0.31641078, 0.9]
5.167585
epoch: 406
[0.34278134, 0.88]
5.866096
epoch: 407
[0.30591437, 0.885]
6.0353794
epoch: 408
[0.4157772, 0.84]
6.2353363
epoch: 409
[0.36566597, 0.87]
5.860565
epoch: 410
[0.3370195, 0.885]
5.919801
epoch: 411
[0.36938614, 0.865]
4.813268
epoch: 412
[0.28845176, 0.905]
5.789108
epoch: 413
[0.30514562, 0.905]
6.541024
epoch: 414
[0.34387735, 0.875]
6.2689924
epoch: 415
[0.33019048, 0.89]
6.3177376
epoch: 416
[0.382718, 0.85]
5.3678837
epoch: 417
[0.29033196, 0.89]
5.8998327
epoch: 418
[0.3340571, 0.89]
6.082842
epoch: 419
[0.28186092, 0.905]
5.704531
epoch: 420
[0.33055016, 0.875]
5.7872186
epoch: 421
[0.37265095, 0.855]
5.510627
epoch: 422
[0.36500987, 0.845]
5.6943593
epoch: 423
[0.36793107, 0.865]
5.507148
epoch: 

epoch: 617
[0.4495594, 0.805]
4.0742803
epoch: 618
[0.40317562, 0.83]
5.462355
epoch: 619
[0.4385071, 0.805]
5.3991156
epoch: 620
[0.37703392, 0.82]
5.187384
epoch: 621
[0.39036942, 0.86]
5.658436
epoch: 622
[0.45596504, 0.785]
4.936857
epoch: 623
[0.41975418, 0.815]
5.639174
epoch: 624
[0.41122177, 0.82]
4.7883024
epoch: 625
[0.39289314, 0.835]
4.646459
epoch: 626
[0.44474807, 0.81]
3.7297745
epoch: 627
[0.40902546, 0.825]
5.557338
epoch: 628
[0.41406646, 0.82]
4.0309367
epoch: 629
[0.37126362, 0.865]
4.631071
epoch: 630
[0.44812283, 0.795]
4.988245
epoch: 631
[0.44688302, 0.81]
5.536669
epoch: 632
[0.42616662, 0.805]
4.8139915
epoch: 633
[0.39655954, 0.85]
4.7611265
epoch: 634
[0.40558204, 0.83]
4.6717815
epoch: 635
[0.39221862, 0.83]
5.778933
epoch: 636
[0.39960128, 0.82]
4.4313545
epoch: 637
[0.41693336, 0.795]
4.1052427
epoch: 638
[0.44022477, 0.795]
5.5372906
epoch: 639
[0.45416224, 0.795]
5.646068
epoch: 640
[0.4749406, 0.77]
4.254106
epoch: 641
[0.4071388, 0.82]
6.7581997
epoch

epoch: 827
[0.45683533, 0.785]
3.771174
epoch: 828
[0.51627004, 0.73]
3.7953358
epoch: 829
[0.45839676, 0.805]
3.8046923
epoch: 830
[0.4566618, 0.805]
3.5811434
epoch: 831
[0.4800391, 0.745]
3.6393363
epoch: 832
[0.50076616, 0.75]
3.9216557
epoch: 833
[0.53684646, 0.69]
4.109511
epoch: 834
[0.46039155, 0.765]
4.262203
epoch: 835
[0.5110618, 0.745]
4.0079937
epoch: 836
[0.48861602, 0.77]
4.8277116
epoch: 837
[0.485831, 0.78]
4.4195924
epoch: 838
[0.5129015, 0.725]
3.588032
epoch: 839
[0.44603166, 0.78]
4.4210777
epoch: 840
[0.48151097, 0.77]
4.609978
epoch: 841
[0.46139094, 0.79]
4.1137185
epoch: 842
[0.4787979, 0.785]
4.647672
epoch: 843
[0.51849127, 0.73]
4.23858
epoch: 844
[0.5222601, 0.71]
4.2247066
epoch: 845
[0.44975403, 0.77]
3.70827
epoch: 846
[0.5265343, 0.735]
3.4490845
epoch: 847
[0.49902573, 0.74]
3.4183044
epoch: 848
[0.484654, 0.765]
4.151201
epoch: 849
[0.5140869, 0.72]
4.717787
epoch: 850
[0.5092462, 0.745]
3.080868
epoch: 851
[0.5423568, 0.705]
3.812803
epoch: 852
[0.55

...

epoch: 19001
[0.644988, 0.63]
0.7603607
epoch: 19002
[0.6591738, 0.565]
0.72091895
epoch: 19003
[0.64691406, 0.59]
0.683498
epoch: 19004
[0.6408267, 0.63]
0.69494176
epoch: 19005
[0.644984, 0.6]
0.6876761
epoch: 19006
[0.6551847, 0.6]
0.702653
epoch: 19007
[0.6923795, 0.515]
0.93926895
epoch: 19008
[0.6555168, 0.615]
0.7780613
epoch: 19009
[0.6576489, 0.58]
0.7871954
epoch: 19010
[0.65188235, 0.61]
0.8520631
epoch: 19011
[0.63478637, 0.65]
0.7854069
epoch: 19012
[0.63570696, 0.65]
0.7429696
epoch: 19013
[0.6652641, 0.595]
0.8042835
epoch: 19014
[0.6399459, 0.65]
0.7997088
epoch: 19015
[0.63606286, 0.63]
0.7838469
epoch: 19016
[0.6404106, 0.64]
0.77255934
epoch: 19017
[0.699525, 0.54]
0.73061013
epoch: 19018
[0.6317849, 0.71]
0.74492526
epoch: 19019
[0.6756752, 0.58]
0.8775592
epoch: 19020
[0.6457544, 0.625]
0.7600493
epoch: 19021
[0.6628555, 0.615]
0.73667204
epoch: 19022
[0.6430909, 0.62]
0.70034003
epoch: 19023
[0.6670224, 0.54]
0.68591386
epoch: 19024
[0.68024665, 0.565]
0.6916049
e

epoch: 19201
[0.68616754, 0.6]
0.76209533
epoch: 19202
[0.66953015, 0.59]
0.7543092
epoch: 19203
[0.66923124, 0.6]
0.7576248
epoch: 19204
[0.6675798, 0.62]
0.77120286
epoch: 19205
[0.67830276, 0.565]
0.74324256
epoch: 19206
[0.67912173, 0.565]
0.7273699
epoch: 19207
[0.68933535, 0.54]
0.6931372
epoch: 19208
[0.69768083, 0.5]
0.7044403
epoch: 19209
[0.6805289, 0.51]
0.6808951
epoch: 19210
[0.68357396, 0.415]
0.67950606
epoch: 19211
[0.6703625, 0.525]
0.6881283
epoch: 19212
[0.67370695, 0.54]
0.6760181
epoch: 19213
[0.6676202, 0.545]
0.68846685
epoch: 19214
[0.6674765, 0.515]
0.7662564
epoch: 19215
[0.6620004, 0.535]
0.6842801
epoch: 19216
[0.67283994, 0.575]
0.6812014
epoch: 19217
[0.6808442, 0.57]
0.7158911
epoch: 19218
[0.6964338, 0.59]
0.71103203
epoch: 19219
[0.66243416, 0.63]
0.98912513
epoch: 19220
[0.6729361, 0.565]
0.7534465
epoch: 19221
[0.67437845, 0.55]
0.7626904
epoch: 19222
[0.6602042, 0.575]
0.715756
epoch: 19223
[0.6570089, 0.645]
0.6747159
epoch: 19224
[0.6618139, 0.63]


epoch: 19399
[0.67892593, 0.535]
0.7364519
epoch: 19400
[0.65998477, 0.595]
0.69248664
epoch: 19401
[0.65590113, 0.575]
0.66804934
epoch: 19402
[0.6476631, 0.605]
0.67294276
epoch: 19403
[0.6610789, 0.55]
0.6831516
epoch: 19404
[0.66036385, 0.565]
0.68694097
epoch: 19405
[0.64434797, 0.61]
0.70396125
epoch: 19406
[0.6434242, 0.57]
0.68565536
epoch: 19407
[0.6538571, 0.575]
0.69004005
epoch: 19408
[0.64865977, 0.595]
0.6941585
epoch: 19409
[0.6587085, 0.565]
0.7362159
epoch: 19410
[0.6862915, 0.52]
0.69838667
epoch: 19411
[0.69319534, 0.505]
0.79190093
epoch: 19412
[0.70011413, 0.53]
0.77911896
epoch: 19413
[0.6950923, 0.515]
0.8602989
epoch: 19414
[0.69854647, 0.515]
0.7821189
epoch: 19415
[0.732556, 0.51]
0.779489
epoch: 19416
[0.68515307, 0.55]
0.75580823
epoch: 19417
[0.6694309, 0.58]
0.74054736
epoch: 19418
[0.7083551, 0.52]
0.7526912
epoch: 19419
[0.69119036, 0.56]
0.73915255
epoch: 19420
[0.67943466, 0.53]
0.89057523
epoch: 19421
[0.680657, 0.53]
0.7874107
epoch: 19422
[0.6991812

epoch: 19601
[0.6263457, 0.53]
0.8551346
epoch: 19602
[0.6453651, 0.58]
0.6755622
epoch: 19603
[0.5914374, 0.645]
1.004553
epoch: 19604
[0.6167198, 0.645]
0.72615355
epoch: 19605
[0.57911944, 0.665]
1.0780864
epoch: 19606
[0.60405594, 0.68]
0.70539284
epoch: 19607
[0.56852734, 0.67]
0.7549225
epoch: 19608
[0.5943396, 0.655]
0.69471407
epoch: 19609
[0.6518309, 0.585]
0.74019104
epoch: 19610
[0.62083113, 0.64]
0.7449254
epoch: 19611
[0.6991357, 0.575]
0.8754557
epoch: 19612
[0.6517792, 0.615]
0.8387444
epoch: 19613
[0.6752178, 0.525]
0.80913234
epoch: 19614
[0.74723893, 0.45]
1.1269915
epoch: 19615
[0.64096963, 0.5]
2.051995
epoch: 19616
[0.5949118, 0.595]
2.931862
epoch: 19617
[0.6105969, 0.6]
3.0856216
epoch: 19618
[0.5074442, 0.665]
4.1106324
epoch: 19619
[0.5437638, 0.665]
2.9091074
epoch: 19620
[0.5878824, 0.76]
0.81624794
epoch: 19621
[0.5892548, 0.71]
0.7123246
epoch: 19622
[0.5939347, 0.68]
0.9053874
epoch: 19623
[0.6141859, 0.635]
0.85072565
epoch: 19624
[0.6203502, 0.655]
0.792

epoch: 19801
[0.6982992, 0.495]
0.7189548
epoch: 19802
[0.69413483, 0.505]
0.7135168
epoch: 19803
[0.6825685, 0.515]
0.7053527
epoch: 19804
[0.7015589, 0.495]
0.7781426
epoch: 19805
[0.69057786, 0.51]
0.76968324
epoch: 19806
[0.67931765, 0.57]
0.76257515
epoch: 19807
[0.69444364, 0.52]
0.73749536
epoch: 19808
[0.68483615, 0.545]
0.6844371
epoch: 19809
[0.70225716, 0.475]
0.72904664
epoch: 19810
[0.6828038, 0.535]
0.7001215
epoch: 19811
[0.68808144, 0.54]
0.6965477
epoch: 19812
[0.70447975, 0.48]
0.7372594
epoch: 19813
[0.7017617, 0.47]
0.8174371
epoch: 19814
[0.6980711, 0.525]
0.7837066
epoch: 19815
[0.6861774, 0.59]
0.77065855
epoch: 19816
[0.68849164, 0.54]
0.7728824
epoch: 19817
[0.69107354, 0.565]
0.7201141
epoch: 19818
[0.6745607, 0.55]
0.72184587
epoch: 19819
[0.6746799, 0.56]
0.6766553
epoch: 19820
[0.67906386, 0.505]
0.65344125
epoch: 19821
[0.6967929, 0.49]
0.6509188
epoch: 19822
[0.70935106, 0.48]
0.7236075
epoch: 19823
[0.6821632, 0.585]
0.79959077
epoch: 19824
[0.6819094, 0